# landcover-hack ODC workshop 2019

This is a test notebook to derive a level 3 classification for select areas in Australia based on the FAO LCCS landcover classification

In [ ]:
import numpy
import xarray
import scipy
from matplotlib import pyplot
from matplotlib import cm
import datacube
from datacube.storage import masking

#import classification script
import lccs_l3

Data input from AWS DEA

In [ ]:
# set up object and bounds

variables_xarray_list = []
dc = datacube.Datacube(app="le_lccs")

pixel_size_x = -25
pixel_size_y = 25

#current area is Ayr
min_x = 1500000.000
max_x = 1600000.000
min_y = -2200000.000
max_y = -2100000.000

out_crs = "EPSG:3577"

#current area is Townsville
max_lon = 147
min_lon = 146.5
min_lat = -19.5
max_lat = -19

start_time = "2015-01-01"
end_time = "2015-12-15"

## 5 layers required for level 3 classification

- vegetat_veg_cat (layer for vegetation/non vegetation)
- aquatic_wat_cat (layer for aquatic/terrestrial
- cultman_agr_cat (layer for cultivated/natural)
- artific_urb_cat (layer for artificial surfaces/bare areas)
- artwatr_wat_cat (layer for artificial water/natural water)

### vegetat_veg_cat

In [ ]:
var_xarray = dc.load(product="fc_percentile_albers_annual", measurements=["PV_PC_10", "NPV_PC_10"],
                     x=(min_x, max_x),
                     y=(min_y, max_y),
                     crs=out_crs,
                     resolution=(pixel_size_x, pixel_size_y),
                     time=(start_time, end_time))

# Check that dataset exists
var_xarray

In [ ]:
var_xarray = dc.load(product="fc_percentile_albers_annual",
                     x=(min_lon, max_lon),
                     y=(min_lat, max_lat),
                     output_crs='epsg:3577',
                     resolution=(pixel_size_x, pixel_size_y),
                     time=(start_time, end_time))

# Check that dataset exists
var_xarray

In [ ]:
# Plot data
var_xarray = masking.mask_invalid_data(var_xarray)
var_xarray.PV_PC_10.plot(figsize=(5, 5), cmap = 'Reds')
var_xarray.NPV_PC_10.plot(figsize=(5, 5), cmap = 'Reds')

In [ ]:
# Create binary layer representing vegetated (1) and non-vegetated (0)
vegetat_veg_cat_ds = ((var_xarray["PV_PC_10"] >= 1) | (var_xarray["NPV_PC_10"] >= 1))

In [ ]:
# Convert to Dataset and add name
vegetat_veg_cat_ds = vegetat_veg_cat_ds.to_dataset(name="vegetat_veg_cat")

In [ ]:
# Plot output
vegetat_veg_cat_ds["vegetat_veg_cat"].plot(figsize=(5, 5))

In [ ]:
# Add to output array
variables_xarray_list.append(vegetat_veg_cat_ds)

### aquatic_wat_cat

In [ ]:
var_xarray = dc.load(product="wofs_annual_summary",measurements=["frequency"], 
                     x=(min_x, max_x),
                     y=(min_y, max_y),
                     crs=out_crs,
                     resolution=(pixel_size_x, pixel_size_y),
                     time=(start_time, end_time))

# Check that dataset exists
var_xarray

In [ ]:
# Plot data
var_xarray = masking.mask_invalid_data(var_xarray)
var_xarray.frequency.plot(figsize=(5, 5), cmap = 'Blues')

In [ ]:
# Create binary layer representing aquatic (1) and terrestrial (0)
aquatic_wat_cat_ds = ((var_xarray["frequency"] >= 0.2))

In [ ]:
# Convert to Dataset and add name
aquatic_wat_cat_ds = aquatic_wat_cat_ds.to_dataset(name="aquatic_wat_cat")

In [ ]:
# Plot output
aquatic_wat_cat_ds["aquatic_wat_cat"].plot(figsize=(5, 5))

In [ ]:
# Add to output array
variables_xarray_list.append(aquatic_wat_cat_ds)

### cultman_agr_cat

### artific_urb_cat

### artwatr_wat_cat

In [ ]:
var_xarray = dc.load(product="geofabric",measurements=["band1"],
                     x=(min_x, max_x),
                     y=(min_y, max_y),
                     crs=out_crs,
                     resolution=(pixel_size_x, pixel_size_y))

# Check that dataset exists
var_xarray

In [ ]:
# Plot data
var_xarray.band1.isel(time=0).plot.imshow(cmap="nipy_spectral")

In [ ]:
# Create binary layer representing artificial water (1) and natural water (0)
artwatr_wat_cat_ds = ((var_xarray["band1"] == 1) | (var_xarray["band1"] == 8))

In [ ]:
# Convert to Dataset and add name
artwatr_wat_cat_ds = artwatr_wat_cat_ds.to_dataset(name="artwatr_wat_cat")

In [ ]:
# Plot output
artwatr_wat_cat_ds["artwatr_wat_cat"].plot(figsize=(5, 5))

In [ ]:
# Add to output array
variables_xarray_list.append(artwatr_wat_cat_ds)

## Classification

The LCCS classificaition is hierachial. The 8 classes are shown below.

| Class name | Code | Numeric code |
|----------------------------------|-----|-----|
| Cultivated Terrestrial Vegetated | A11 | 111 |
| Natural Terrestrial Vegetated | A12 | 112 |
| Cultivated Aquatic Vegetated | A23 | 123 |
| Natural Aquatic Vegetated | A24 | 124 |
| Artificial Surface | B15 | 215 |
| Natural Surface | B16 | 216 |
| Artificial Water | B27 | 227 |
| Natural Water | B28 | 228 |


In [ ]:
# Merge to a single dataframe
classification_data = xarray.merge(variables_xarray_list)

In [ ]:
# Apply Level 3 classification using separate function. Works through in three stages
level1, level2, level3 = lccs_l3.classify_lccs_level3(classification_data)

In [ ]:
# Save classification values back to xarray
out_class_xarray = xarray.Dataset({"level1" : (classification_data["vegetat_veg_cat"].dims, level1),
                                   "level2" : (classification_data["vegetat_veg_cat"].dims, level2),
                                   "level3" : (classification_data["vegetat_veg_cat"].dims, level3)})
classification_data = xarray.merge([classification_data, out_class_xarray])
classification_data

col_level2 = cm.get_cmap("Set1", 2)
classification_data.level2.plot(cmap=(col_level2))

In [ ]:
#To check the results for level 3 use colour_lccs_level3 to get the colour scheme.
red, green, blue, alpha = lccs_l3.colour_lccs_level3(level3[0])
pyplot.imshow(numpy.dstack([red, green, blue, alpha]))